In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-269498
Azure region: eastus2
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-269498


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "test-alll"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

     
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
# from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': uniform(0.1, 4.0),
    '--max_iter': choice(50, 100, 150, 200, 250, 300)
})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2, delay_evaluation=6)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig('./', script='train.py', compute_target=cluster, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=8,
                                    max_concurrent_runs=4,
                                    run_config=src)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

2024-11-05 03:52:36.620986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 03:52:36.643322: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 03:52:36.649927: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 03:52:36.666264: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-05 03:52:37.823788: W tensorflow/comp

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_2e477c33-057e-4fb4-a711-fad47c57378b
Web View: https://ml.azure.com/runs/HD_2e477c33-057e-4fb4-a711-fad47c57378b?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-269498/workspaces/quick-starts-ws-269498&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-11-05T03:52:34.0233267Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2024-11-05T03:52:34.3314069Z][SCHEDULER][INFO]Scheduling job, id='HD_2e477c33-057e-4fb4-a711-fad47c57378b_0' 
[2024-11-05T03:52:34.4556454Z][SCHEDULER][INFO]Scheduling job, id='HD_2e477c33-057e-4fb4-a711-fad47c57378b_3' 
[2024-11-05T03:52:34.4579290Z][SCHEDULER][INFO]Scheduling job, id='HD_2e477c33-057e-4fb4-a711-fad47c57378b_1' 
[2024-11-05T03:52:34.4568663Z][SCHEDULER][INFO]Scheduling job, id='HD_2e477c33-057e-4fb4-a711-fad47c57378b_2' 
[2024-11-05T03:52:34.9658268Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_2e477c33-057e-4fb4-a711-fad47c57378b_2' 
[2024-11-05T03:5

{'runId': 'HD_2e477c33-057e-4fb4-a711-fad47c57378b',
 'target': 'test-alll',
 'status': 'Completed',
 'startTimeUtc': '2024-11-05T03:52:32.396708Z',
 'endTimeUtc': '2024-11-05T03:55:34.976868Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'dcdaac70-4f8c-4d29-83d3-d0077e97f910',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_2e477c33-057e-4fb4-a711-fad47c57378b_4',
  'score': '0.9092564491654022',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_2e477c33-057e-4fb4-a711-fad47c57378b_4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-tra

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_2e477c33-057e-4fb4-a711-fad47c57378b_4,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
print(best_run)

Run(Experiment: udacity-project,
Id: HD_2e477c33-057e-4fb4-a711-fad47c57378b_4,
Type: azureml.scriptrun,
Status: Completed)


In [7]:
best_run_metrics = best_run.get_metrics()
best_run_parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])

print(best_run_parameter_values)

Best Run Id:  HD_2e477c33-057e-4fb4-a711-fad47c57378b_4
Accuracy: 0.9092564491654022
['--C', '1.3311456981265843', '--max_iter', '250']


In [8]:
# joblib.dump(value=best_run, filename='./model.joblib')

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [10]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=cluster,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [12]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()

Submitting remote run.
No run_configuration provided, running on test-alll with default configuration
Running on remote compute: test-alll


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_12955c4b-33ac-4bb3-a161-7aa7a24bd986,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [13]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_run_job, best_automl_model = automl_run.get_output()
joblib.dump(value=best_automl_model, filename='automl_model.joblib')

['automl_model.joblib']

In [14]:
best_automl_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/uda/code/Users/odl_user_269498')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassifier(classification_labels=ar...ter=1000, penalty=None, power_t=0.7777777777777777, tol=0.0001))]))], flatten_transform=False, weights=[0.14285714285714285, 0.2857142857142857, 0.07142857142857142, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [15]:
automl_run_job

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_12955c4b-33ac-4bb3-a161-7aa7a24bd986_24,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
cluster.delete()

In [21]:
cluster

AmlCompute(workspace=Workspace.create(name='quick-starts-ws-269498', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-269498'), name=test-alll, id=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourceGroups/aml-quickstarts-269498/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-269498/computes/test-alll, type=AmlCompute, provisioning_state=Deleting, location=eastus2, tags={})

In [23]:
cluster.list(ws)

[{
   "id": "/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourceGroups/aml-quickstarts-269498/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-269498/computes/uda",
   "name": "uda",
   "location": "eastus2",
   "tags": {},
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "eastus2",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "Standard_DS3_v2",
       "applications": [
         {
           "displayName": "Jupyter",
           "endpointUri": "https://uda.eastus2.instances.azureml.ms/tree/"
         },
         {
           "displayName": "Jupyter Lab",
           "endpointUri": "https://uda.eastus2.instances.azureml.ms/lab"
         }
       ],
       "connectivityEndpoints": {
         "publicIpAddress": "172.200.233.161",
         "privateIpAddress": "10.0.0.4"
       },
       "sshSettings": {
         "s